In [1]:
pdb_file = "/Users/madilyn/Projects/repos/forcefields/pdbfiles/P3HT.pdb"
mol2file = '/Users/madilyn/Projects/repos/forcefields/mol2files/p3ht_md.mol2'
xml_filename = "/Users/madilyn/Projects/repos/forcefields/xml_files/p3ht.xml"
typed_mol2 = "/Users/madilyn/Projects/repos/forcefields/mol2files/p3ht_typed.pdb"

In [2]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml
from bondwalk import bond_walk
from bondwalk.bond_walk import MadAtom, MadBond, BondWalker

import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import mbuild as mb
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import numpy as np
import torch
from openff.toolkit.topology import Molecule
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
import hoomd
import gsd.hoomd
import matplotlib.pyplot as plt
import rdkit
from rdkit import Chem

import os
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")
    
## creates the openff molecule object as well as saving a pdb file for renaming later
def openff_Molecule(mol2_file):
    rdmol = Chem.MolFromMol2File(mol2file)
    from_rdmol = Molecule.from_rdkit(rdmol)
    atom_list = [a for a in from_rdmol.atoms if 1 not in [a.atomic_number]]
    for atom in atom_list:
        atom.formal_charge = 0
    b = BondWalker(from_rdmol)
    comp = b.fill_in_bonds()
    from_rdmol.to_file(pdb_file,file_format="pdb")
    return comp



molecule = openff_Molecule(mol2_file=mol2file)

#Running the espaloma code

molecule_graph = esp.Graph(molecule)

espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph,charge_method="nn")

# Store the results for each in something more accessible
pair_forces = openmm_system.getForces()[1]
angle_forces = openmm_system.getForces()[3]
bond_forces = openmm_system.getForces()[2]
torsion_forces = openmm_system.getForces()[0]



# get a parmed structure from openmm 
import parmed as pmd
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()

structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)


for i in range(len(molecule.atoms)):
    if molecule.atoms[i].atomic_number == 6:
        molecule.atoms[i].name = 'C'
    if molecule.atoms[i].atomic_number == 1:
        molecule.atoms[i].name = 'H'
    if molecule.atoms[i].atomic_number == 7:
        molecule.atoms[i].name = 'N'
    if molecule.atoms[i].atomic_number == 16:
        molecule.atoms[i].name = 'S'
    if molecule.atoms[i].atomic_number == 8:
        molecule.atoms[i].name = 'O'
    if molecule.atoms[i].atomic_number == 9:
        molecule.atoms[i].name = 'F'

import networkx  as nx
Gopenmm = nx.Graph()
Gparmed = nx.Graph()
#openmm:
for i in range(bond_forces.getNumBonds()):
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
#parmed
for b in structure.bonds:
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)
    
particle_types = []
type_map = dict()

#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for i in range(pair_forces.getNumParticles()):
        pair_parms = pair_forces.getParticleParameters(index=i)
        sigma = pair_parms[1]/pair_parms[1].unit
        epsilon = pair_parms[2]/pair_parms[2].unit
        if (sigma, epsilon) not in particle_types: 
            particle_types.append((sigma, epsilon))
        type_map[molecule.atoms[i].molecule_atom_index] = "".join([molecule.atoms[i].name , 
                                                                   str(particle_types.index((sigma, epsilon)))])
        
# Rename the particle types so that they match the xml file
# This is needed when we aren't using SMARTS matching with Foyer.

comp_rename = mb.load(pdb_file)

for index in type_map:
    #print(index, type_map[index],comp_rename[index].name)
    comp_rename[index].name = type_map[index]

#Create your dictionaries:

bond_types = []
bond_dict = dict() 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':k,'l0':l0}


angle_types = []
angle_dict = dict()

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit  
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':k,'t0':t0}



dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    if i%6==0:
        periodicity=[]
        phase = []
        k = []
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity.append(dihedral_parms[4])  
    phase.append( dihedral_parms[5]/dihedral_parms[5].unit)
    k.append(dihedral_parms[6]/dihedral_parms[6].unit)
    dt = (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]])
   

    if periodicity[-1]==6:
        dihedral_dict[dt] = {'periodicity':periodicity,'k':k,'phase':phase}


nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


Done!


In [3]:
## Save the forcefield XML file for future use, so that we don't have to repeat the espaloma process everytime
#mbuild_to_foyer_xml(
#    file_name=xml_filename, #change this to whatever you want to save your xml file as
#    compound=comp_rename,
#    bond_params=bond_dict,
#    angle_params=angle_dict,
#    dihedral_params=dihedral_dict,
#    dihedral_type="periodic",
#    non_bonded_params=nonbonded_dict,
#    combining_rule="geometric",
#    name="",
#    version="",
#    coulomb14scale=1.0,
#    lj14scale=1.0)
#

In [4]:
#Save the mb.Compound with the new atom type names for future use.
comp_rename.save(typed_mol2, overwrite=True) #change this to match your molecule name. 

## Children Issue 

at this point we've successfully parameterized our monomer and dimer and resaved them with the correct atom types to match the forcefield file. We just have to determine how to extract the monomer so we can build polymers from it. We have to build the polymers from the typed monomer so it'll have the correct atom types to be able to apply the forcefield. 

In [5]:
p3ht_typed = mb.load(typed_mol2)
p3ht_typed.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

try to access the mbuild children to pull just the monomer out to use for polymer building:

the p3ht_typed file should have two children (the dimer and the monomer), however it only has one. 
We either have to figure out how to rmeove the dimer from this mbuild compound or extract the monomer 
to use for polymer building 

In [6]:
len(p3ht_typed.children)

1

Theres not an easy way to determine which atoms are part of the monomer and which are part of the dimer, but maybe if we could figure out where the origin lies we could determine that all the dimers coordinates are negative, for instance, and delete all the atoms with negative coordinates?

When we save the dimer and monomer together (done in the polymer builder ipynb) we could move the dimer into negative coordinates and the monomer into positive?

In [7]:
#I saved the dimer in negative coordinates and monomer in positive
#This tells me that indices 0-10 and 33-48 belong to the monomer and the rest belong to the dimer 

for i in range(p3ht_typed.n_particles):
    print(i,p3ht_typed[i].name,p3ht_typed[i].xyz)

0 C0 [[3.41659999 3.06710005 3.02649999]]
1 C0 [[3.27789998 3.0539     2.96429992]]
2 C0 [[3.18530011 2.96930003 3.05259991]]
3 C0 [[3.04649997 2.95639992 2.98889995]]
4 C0 [[2.9533     2.87059999 3.07579994]]
5 C0 [[2.81480002 2.85330009 3.01160002]]
6 C1 [[2.7420001  2.9842     2.99670005]]
7 C1 [[2.70549989 3.06419992 3.10450006]]
8 S2 [[2.62700009 3.20029998 3.04489994]]
9 C1 [[2.63840008 3.15919995 2.8822999 ]]
10 C1 [[2.70409989 3.03740001 2.87249994]]
11 C0 [[-2.58380008 -2.94799995 -2.94989991]]
12 C0 [[-2.72550011 -2.97009993 -3.00139999]]
13 C0 [[-2.81369996 -3.03900003 -2.89630008]]
14 C0 [[-2.9546001  -3.06270003 -2.95169997]]
15 C0 [[-3.04870009 -3.125      -2.84640002]]
16 C0 [[-3.1875999  -3.1552999  -2.90560007]]
17 C1 [[-3.2500999  -3.0309     -2.96219993]]
18 C1 [[-3.29060006 -2.92190003 -2.88610005]]
19 S2 [[-3.34509993 -2.80279994 -2.9914999 ]]
20 C1 [[-3.31489992 -2.88680005 -3.13470006]]
21 C1 [[-3.2665     -3.01259995 -3.0999999 ]]
22 C0 [[-2.60770011 -2.79299998

This is where I'm looking for ideas:
1) either pull the monomer out and create a new mbuild compound of the monomer
2) delete the dimer coordinates 
3) only parameterize the dimer, then remove the dimer bond and create a monomer from the dimer by deteting half of the dimer